# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = '/Users/anncheng/Documents/GitHub/4th-ML100Days/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape
# print(df_train)
# print(df_test)
df_train


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
df.dtypes
# type(df.dtypes)為 pandas.core.series.Series

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [3]:
# 秀出資料欄位的類型與數量

# 使用df.dtypes.reset_index()是為了將df.dtypes轉成dataframe，而轉成dataframe的目的是此cell倒數第二行才能group by
# # df.dtypes.reset_index()
dtype_df = df.dtypes.reset_index()
dtype_df
# type(dtype_df)為 dataframe
# 由dtype_df.columns或dtype_df可知dtype_df的column名稱為index與0，所以接下來要改column名稱

dtype_df.columns = ["Count", "Column Type"]
dtype_df
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
# type(object_features) 為 list
# 有個 a = [1,2,0.3,True]，for b in a: print(b)
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        # list.append(obj)中obj表在列表中要追加的對象。參考資料http://tw.gitbook.net/python/list_append.html        
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
        # print(int_features, float_features, object_features)
        
# 因為int_features是list，而list計算len的方式是看中括號裡面有幾個東西，如 a = ['1','2','b',True]則len(a)為4   
# f‘{}’中{}裡的內容會被換掉
# \n是換一行，但因為print本來就分行寫，所以如果又再加上\n結果看起來就是變空一行。可嘗試print('abc\ndefg')就是換一行不是空一行。
print(f'{len(int_features)} Integer Features : {int_features}')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')
# type(object_features) 仍為 list

3 Integer Features : ['Pclass', 'SibSp', 'Parch']
2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  
專家Ans:只有 object 的 mean() 與 max() 會有問題，原因是文字無法進行平均或比較大小，但我們可以發現，object 仍有部分欄位可傳回最大值，可以知道 object 欄位超過一定長度時會當成字串，此時會以字典順序做大小比較

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?
專家Ans: 舉例 : 複數 (a+bi)，因為需要兩個數字表示，雖然概念上是數值，但因為無法比較大小，因此只能歸類為"類別型欄位"
不論其他更奇怪的例子，都可以歸類為類別型欄位，因此類別型欄位應該最難處理

## 作業1_mean


In [5]:
# 例 : 整數 (int) 特徵取平均 (mean)
# type(int_features)為list，type(df[int_features])為dataframe
int_features_mean = df[int_features].mean()
float_features_mean = df[float_features].mean()
object_features_mean = df[object_features].mean()

"""
嘗試用其他方式取mean，但都失敗
int_features.mean()
array_int_features = np.array(int_features)
type(array_int_features)
matrix_array_int_features = np.mat(array_int_features)
type(matrix_array_int_features)
np.mean(matrix_array_int_features)
"""

print(f'{int_features_mean}\n')
print(f'{float_features_mean}\n')
# object 非數值，應該沒有平均可以取，與取標準差的結果一樣(後面cell有嘗試取標準差)
print(f'{object_features_mean}\n')
#  type(object_features_mean) 為 pandas.core.series.Series

Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

Age     29.881138
Fare    33.295479
dtype: float64

Series([], dtype: float64)



## 作業1_max

In [6]:
int_features_max = df[int_features].max()
float_features_max = df[float_features].max()
object_features_max = df[object_features].max()

print(f'{int_features_max}\n')
print(f'{float_features_max}\n')
# object 非數值，取出來應該沒有最大值，雖然結果有return一些內容但不確定以什麼條件去評斷
print(f'{object_features_max}\n')

Pclass    3
SibSp     8
Parch     9
dtype: int64

Age      80.0000
Fare    512.3292
dtype: float64

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object



## 作業1_unique

In [7]:
int_features_unique = df[int_features].nunique()
float_features_unique = df[float_features].nunique()
object_features_unique = df[object_features].nunique()

print(f'{int_features_unique}\n')
print(f'{float_features_unique}\n')
print(f'{object_features_unique}\n')

Pclass    3
SibSp     7
Parch     8
dtype: int64

Age      98
Fare    281
dtype: int64

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64



## 作業1_取mean, max, unique以外的操作_std

In [8]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
"""
Your Code Here
"""
#  std 為標準差
int_features_std = df[int_features].std()
float_features_std = df[float_features].std()
object_features_std = df[object_features].std()


print(f'{int_features_std}\n')
print(f'{float_features_std}\n')
# object 非數值，應該沒有平均可以取，結果與取平均數的結果一樣
print(f'{object_features_std}\n')

Pclass    0.837836
SibSp     1.041658
Parch     0.865560
dtype: float64

Age     14.413493
Fare    51.758668
dtype: float64

Series([], dtype: float64)



## 作業2

音樂或影像為 int / float / object / 時間 以外的資料類型。
int / float / object 最難處理的是物件類型的資料